In [251]:
import torch

a = torch.tensor([
    [3, 2, 1],
    [1, 2, 3]
])

idx = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2])

# torch.argsort(idx, ascending=True)

@torch.jit.script
def stable_arg_sort_long(arr):
    delta = torch.linspace(0, 0.99, arr.shape[0])
    return torch.argsort(arr + delta)

stable_arg_sort_long(idx)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [248]:
idx = torch.tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2])

delta = torch.linspace(0, 0.99, idx.shape[0])
torch.argsort(idx + delta)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [213]:
@torch.jit.script
def unique_with_counts(idx, grouped: Dict[int, int]):
    for x in idx:
        if x.item() not in grouped:
            grouped[x.item()] = 1
        else:
            grouped[x.item()] += 1
            
    counts = torch.zeros(len(grouped), dtype=torch.long)
    values = torch.empty(len(grouped))
    for i, (k, v) in enumerate(grouped.items()):
        values[i] = k
        counts[i] = v
    a = torch.argsort(values)
    
    return values[a], counts[a]

@torch.jit.script
def scatter_group(x: torch.Tensor, idx: torch.Tensor, d: Dict[int, int]) -> Tuple[torch.Tensor, List[torch.Tensor]]:
    x = x[toarch.argsort(idx)]
    groups, b = unique_with_counts(idx, d)
    i_a = 0
    arr_list = []
    for i_b in b:
        arr_list.append(x[i_a:i_a + i_b.item()])
        i_a += i_b.item()
    return groups, arr_list


l = 1000
idx = torch.randint(0, l, torch.Size([l]), dtype=torch.long)
x = torch.randn(l)

scatter_group(x, idx, {})
# idx
%timeit -n 10 scatter_group(x, idx, {})
# print(idx)
# unique_with_counts(idx, {})
l = 10
idx = torch.randint(0, l, torch.Size([l]), dtype=torch.long)
x = torch.randn(l)
print(x)
print(idx)

b = scatter_group(x, idx, {})
b

5.11 ms ± 36.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
tensor([ 0.0798, -1.5222,  0.6170, -1.3649, -0.0736,  0.2720,  1.6178, -1.3368,
        -1.2623, -0.0814])
tensor([4, 1, 5, 1, 1, 3, 2, 2, 5, 6])


(tensor([1., 2., 3., 4., 5., 6.]),
 [tensor([-1.5222, -1.3649, -0.0736]),
  tensor([ 1.6178, -1.3368]),
  tensor([0.2720]),
  tensor([0.0798]),
  tensor([ 0.6170, -1.2623]),
  tensor([-0.0814])])

In [215]:
def graph_batch_to_nx(batch):
    gidx_1, node_attrs = scatter_group(batch.x, batch.node_idx)
    gidx_2, edge_attrs = scatter_group(batch.e, batch.edge_idx)
    for k, n in zip()

In [18]:
torch.unique(arr, return_counts=True)

(tensor([0, 1, 2, 3]), tensor([2, 1, 1, 1]))

In [19]:
torch.split(arr, [2, 1, 1, 1])

(tensor([0, 0]), tensor([1]), tensor([2]), tensor([3]))